In [1]:
import numpy as np
import pandas as pd
import orangecontrib.associate.fpgrowth as oaf
import datetime
import argparse
#parser = argparse.ArgumentParser()
#parser.add_argument('--train_dir', type=str, default='./trainDataHJ.txt')
#parser.add_argument('--rules_dir', type=str, default='./rulesDataHJ.txt')
#parser.add_argument('--support', type=float, default=0.05)
#parser.add_argument('--confidence',type=float,default=0.2)
#args = parser.parse_args()

In [2]:
y=np.random.randint(1,6,[1000,])
hum=np.random.randint(0,101,[1000,])
tem=np.random.uniform(-50.0,40.0,[1000,])
is_gas=np.random.randint(0,2,1000)
per_con=np.random.randint(0,101,[1000,])
is_metal=np.random.randint(0,2,1000)
mach_cat=np.random.randint(1,4,[1000,])
meth_cat=np.random.randint(1,4,[1000,])


In [3]:
df=pd.DataFrame({
    "hum":hum,
    "tem":tem,
    "is_gas":is_gas,
    "per_con":per_con,
    "is_metal":is_metal,
    "mach_cat":mach_cat,
    "meth_cat":meth_cat,
    "time":y
})

In [5]:
#df.to_csv('trainDataHJ.csv',index=False)

In [9]:
def data_pre(path):
    df=pd.read_csv(path)
    #iscretization
    bins=[0,2,5,11]
    df['d_age']=pd.cut(df['d_age'],bins,right=False,labels=['1','2','3'])
    bins=[0,2,5,11]
    df['busy_time']=pd.cut(df['busy_time'],bins,right=False,labels=['1','2','3'])
    bins=[0,2,5,11]
    df['free_time']=pd.cut(df['free_time'],bins,right=False,labels=['1','2','3'])
    #type trans
    df['on_time']=df['on_time'].apply(str)
    df['start_time_type']=df['start_time_type'].apply(str)
    df['close_time_type']=df['close_time_type'].apply(str)
    df['d_type']=df['d_type'].apply(str)
    df['is_fault']=df['is_fault'].apply(str)
    #one_hot
    df=pd.get_dummies(df)
    strDecode=df.columns.values.tolist()
    data=df.values
    data.dtype=bool
    return data,strDecode

In [4]:
def model(data,support=0.05,confidence=0.2):
    fre_ite=dict(oaf.frequent_itemsets(data,support))  #这里设置置信度
    rules = oaf.association_rules(fre_ite,confidence)
    result=list(rules)
    return result

In [8]:
def dealResult(rules,strDecode):
    returnRules = []
    for i in rules:
        j= list(i[1])
        if len(j)==1 and strDecode[j[0]].startswith('is_fault') :
            temStr = [];
            temp=''
            for k in i[0]:
                temp = temp+strDecode[k]+'&'
            temp = temp[:-1]
            temp = temp + ' ==> '
            temp = temp +strDecode[j[0]] 
            temStr.append(temp)
            temStr.append(i[2])
            temStr.append(i[3])
            returnRules.append(temStr)
    returnRules=list(sorted(returnRules,key=lambda x:(-x[2],-x[1])))
    return returnRules
    


In [10]:
support=0.05
confidence=0.2
train_dir='trainDataGZ.txt'
rules_dir='rulesDataGZ.txt'
data,strDecode=data_pre(train_dir)
result=model(data,support,confidence)
printRules= dealResult(result,strDecode)
file=open(rules_dir,'w')
print('Rules'+'\t'+'support'+'\t'+'confidence'+'\n')
file.write('Rules'+'\t'+'support'+'\t'+'confidence'+'\n')
for line in printRules:
    s='\t'.join([str(i) for i in line])
    print(s+'\n')
    file.write(s+'\n')
file.close()
    

Rules	support	confidence

start_time_type_3&busy_time_3 ==> is_fault_0	52	0.65

start_time_type_4 ==> is_fault_1	74	0.6379310344827587

free_time_3&start_time_type_3 ==> is_fault_0	50	0.6329113924050633

start_time_type_3 ==> is_fault_0	78	0.6290322580645161

close_time_type_7 ==> is_fault_1	81	0.627906976744186

on_time_3&d_type_1 ==> is_fault_0	63	0.6176470588235294

d_type_3&free_time_2 ==> is_fault_1	53	0.6091954022988506

d_type_2&d_age_3&busy_time_3 ==> is_fault_1	74	0.6065573770491803

d_age_1 ==> is_fault_1	66	0.6

busy_time_3&start_time_type_5 ==> is_fault_1	52	0.5977011494252874

d_age_2&d_type_3 ==> is_fault_1	55	0.5913978494623656

free_time_3&busy_time_3&d_type_3 ==> is_fault_1	63	0.5887850467289719

busy_time_3&d_type_3 ==> is_fault_1	105	0.5833333333333334

on_time_2&d_age_3&busy_time_3 ==> is_fault_1	67	0.5826086956521739

d_age_2&free_time_2 ==> is_fault_1	53	0.5824175824175825

close_time_type_6&busy_time_3 ==> is_fault_1	53	0.5824175824175825

free_time_3&d_age_3&d_t